In [1]:
import numpy as np
#import gym
import gymnasium as gym
import random
import time
from warnings import filterwarnings

#filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool8` is a deprecated alias')

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="ansi", desc=None)
#env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="human", desc=None)

action_space_size = env.action_space.n
state_space_size = env.observation_space.n
#print(action_space_size)
#print(state_space_size)

q_table = np.zeros((state_space_size, action_space_size))

num_episodes = 10000
max_steps_per_episode = 3000

learning_rate = 0.11
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 0.99
min_exploration_rate = 0.01
exploration_decay_rate = 0.002

rewards_all_episodes = []

# Q-Learnig Algorithm
for episode in range(num_episodes):
    print("*** Episode ",  episode+1, " ***", end="\r")
    state = env.reset()[0]

    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode):

        # Exploration-Exploitation Trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state, :])
        else:
            action = env.action_space.sample()
        new_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # Update Q-Table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        state = new_state
        rewards_current_episode += reward

        if done == True:
            break

    # Exploration rate decay
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)

    rewards_all_episodes.append(rewards_current_episode)

# Calsulate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000
print("*** Average reward per thousand episodes ***\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

# Print updated Q-Table
print("\n\n*** Updated Q-Table ***\n")
print(q_table)
input("Press Enter to continue...")

num_episodes = 5
max_steps_per_episode = 99

for episode in range(num_episodes):
    state = env.reset()[0]

    done = False

    for step in range(max_steps_per_episode):
        print("\033c")
        print("*** Episode ",  episode+1, " ***\n", step)
        env.render()
        print(env.render())
        time.sleep(0.1)

        action = np.argmax(q_table[state, :])
        new_state, reward, done, unknown, info = env.step(action)

        # Update Q-Table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        state = new_state
        rewards_current_episode += reward

        if done == True:
            print("\033c")
            print("*** Episode ",  episode+1, " ***\n")
            env.render()
            print(env.render())
            if reward == 1:
                print("*** You reached the goal! ***")
                time.sleep(3)
            else:
                print("*** You fell through a hole! ***")
                time.sleep(3)
            #clear_output(wait=True)
            break

        if step == max_steps_per_episode - 1:
            print("*** You failed to complete the task! ***")
            time.sleep(3)

*** Average reward per thousand episodes ***

1000 :  0.11000000000000008
2000 :  0.4930000000000004
3000 :  0.6470000000000005
4000 :  0.6550000000000005
5000 :  0.6930000000000005
6000 :  0.6870000000000005
7000 :  0.6650000000000005
8000 :  0.6500000000000005
9000 :  0.6800000000000005
10000 :  0.6750000000000005


*** Updated Q-Table ***

[[0.57993467 0.51267192 0.50222465 0.50109174]
 [0.34539723 0.2906756  0.35076289 0.52302719]
 [0.43173201 0.40667045 0.37929596 0.46030601]
 [0.28623885 0.32288406 0.24074335 0.44529437]
 [0.60510052 0.35078577 0.42644849 0.30428772]
 [0.         0.         0.         0.        ]
 [0.13017167 0.06997281 0.29045955 0.17154257]
 [0.         0.         0.         0.        ]
 [0.43117763 0.44361547 0.47607988 0.63116439]
 [0.4383851  0.68790239 0.39833583 0.35594036]
 [0.60190807 0.37221207 0.35158131 0.30743901]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.53678261 0.31189016 0.82230316 0.5893341